**Evaluación usando LogicSim+**

Notas:
1. gpt-oss tiene predicados incompletos. Para la siguiente ronda de evaluación va a ser necesario darle más tokens de generación.
2. En general los LLMs tienen traducciones de mayor similitud, tienen problemas con los predicados que usan múltiples variables. 
3. LogicSim necesita comparar a fondo los predicados, no solo la cantidad de los mismos.
4. Hay que extender LogicSim para que funcione en los distintos puntos de evaluación. (AHORITA HACEMOS ESTE PEDO.)

In [ ]:
from datasets import load_dataset
import pandas as pd
import re
import os
from nltk.inference.prover9 import *

# Cluster de Helena. Bendita sea la doctora en verdad.
os.environ['PROVER9'] = '/media/discoexterno/francisco/Prover9/bin'

argument = (
    'some x.(Movie(x) & not HappyEnding(x))',
    [
        'not all x.(Movie(x) -> HappyEnding(x))',
        'Movie(titanic)',
        'not HappyEnding(titanic)',
        'Movie(lionKing)',
        'HappyEnding(lionKing)',
    ]
)

def prove(argument):
    goal, assumptions = argument
    g = Expression.fromstring(goal)
    alist = [Expression.fromstring(a) for a in assumptions]
    p = Prover9Command(g, assumptions=alist).prove()
    return p


#new_folio = load_dataset('fol-autoformalization/FOLIO', token = hf_token)
#new_folio

# Amoxtli
#tran_ds = pd.read_csv(r'/home/flopezp/Kurosagol/Ongoing/datasets/gpt_qwen_translations.csv')

# Server Dra. Helena (bendiciones)
tran_ds = pd.read_csv(r'/media/discoexterno/francisco/Kurosagol/Ongoing/datasets/gpt_qwen_translations_8ktokens.csv')
tran_ds = tran_ds.drop(columns=["Unnamed: 0"])
tran_ds.head()

/home/flopez/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,story_id,premises,premises-FOL,conclusion,conclusion-FOL,label,example_id,gpt Translations,qwen Translations
0,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete.,Athlete(stephenCurry),Uncertain,828,∀x (ProSoccerPlayer(x) → ¬PlayBasketball(x))\n...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬Profess...
1,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete and a professional...,Athlete(stephenCurry) ∧ Professional(stephenCu...,False,829,∀x (SoccerPlayer(x) → ¬BasketballPlayer(x)) \...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬PlaysBa...
2,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,"If Stephen Curry is a professional centerback,...",(Professional(stephenCurry) ∧ CenterBack(steph...,True,830,FOL Premises:\n∃x (Athlete(x) ∧ CenterBack(x))...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬Profess...
3,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",Tyler joined the student government in high sc...,"JoinIn(tyler, studentGovernment, highSchool)",Uncertain,947,∀x (SG(x) → Opportunity(x)) \n∀x (Alone(x) → ...,\n\n∀x (JoinStudentGovernment(x) → Opportuniti...
4,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",If Tyler either both stays alone in high schoo...,"¬(StayAloneIn(tyler, highSchool) ⊕ (Well-adjus...",True,948,∀x (JoinSG(x) → ManyOpportunities(x)) \n∀x (S...,\n\n∀x (JoinSG(x) → Opportunities(x)) \n∀x (S...


In [2]:
# LogicSim+

def get_nice_dict(regex, instance, const):
	"""
		A partir de una regex, extrae todos los valores encontrados y los formatea para tener un diccionario con apariciones.

		regex = str ; Expresión regular a buscar.
		const = bool ; Determina si estamos analizando constantes. Esto permite que se realice un filtro extra en el código.
	"""
	lista = []
	for _ in instance:
		aux = re.finditer(regex, _)
		for expression in aux:
			if const:
				regex_list = expression.group()[1:-1]
				regex_list = regex_list.split(',')
				for j in regex_list:
					if len(j) > 1:
						constant = re.sub(' ', '', j)
						lista.append(constant)
			else:
				lista.append(expression.group())

	set_set = list(set(lista))
	dict_dict = {}
	for _ in set_set:
		dict_dict["{}".format(_)] = lista.count(_)

	return dict_dict


def extract_info(ds_answer, llm):
	"""
		Limpia la respuesta de un dataset y extrae las premisas necesarias para calcular LogicSim.

		llm = Bool ; Señala si se va a evaluar la respuesta generada por un LLM.
	"""
	instance = ds_answer.split('\n')
	
    # Este bloque permite limpiar las respuestas que siguen en exceso el prompt inicial.
	if llm: 
		for i in range(len(instance)):
			instance[i] = re.sub('(::)+([ A-z.]+)', '', instance[i])
			instance[i] = re.sub('(:::)+([ A-z.]+)', '', instance[i])
			instance[i] = re.sub('(  )+', '', instance[i])

	pred_dict = get_nice_dict(r'[A-z]+\(([A-z]+(,? [A-z]+)*)\)', instance, False)
	const_dict = get_nice_dict(r'(\([A-z]+(\, [A-z]+){0,}\))', instance, True)
	logop_dict = get_nice_dict(r'[∀∧→⊕¬∨∃]+', instance, False)

	# len(instance) = Cantidad de premisas
	# pred_dict = Diccionario con cantidad de predicados. DE AQUÍ SE OBTIENE APARICIONES TOTALES Y CANTIDAD DE PREDICADOS
	# constant_dict =  Diccionario con cantidad de constantes. DE AQUÍ SE OBTIENE APARICIONES TOTALES Y CANTIDAD DE CONSTANTES
	# logop_dict =  Diccionario con cantidad de operadores y cuantificadores. DE AQUÍ SE OBTIENE APARICIONES TOTALES Y CANTIDAD DE CUANTIFICADORES
	return pred_dict, const_dict, logop_dict, len(instance)


def total_apps(dict1, preds):
    """
        Obtiene:
            1) La cantidad total de apariciones de un valor (Const/Pred/LogOps)
            2) La cantidad de constantes/predicados/logops distintos.

        dict1 = dict ; El diccionario obtenido previamente.
        preds = bool ; Valor booleano que permite realizar un filtro sobre los predicados.
    """
    if preds:
        lista_chida_aux = [re.search(r'([A-z]+\()', _).group() for _ in dict1.keys()]
        lista_chida = list(set(lista_chida_aux))

        # Guardamos la cantidad de apariciones en el diccionario previo.
        aux_dict = {"{}".format(_):0 for _ in lista_chida}
        for key in dict1.keys():
            for _ in lista_chida:
                if _ in key:
                    aux_dict["{}".format(_)] += dict1[key]
        
        if preds:
            aux_dict = {"{}".format(_[:-1]): aux_dict[_] for _ in aux_dict}
    else:
        aux_dict = dict1
    
    total_value = 0
    for _ in aux_dict:
        total_value += aux_dict[_]

    #total_value = Apariciones totales
    #len(aux_dict) = Cantidad de [VALUE] distinto.
    return total_value, len(aux_dict)


def logic_sim_plus_individual(llm_ans, folio_ans):
    """
        Extrae los valores distintos y valores totales de cada instancia. 
    """
    pred_llm, const_llm, logops_llm, len_llm = extract_info(llm_ans, False)
    pred_folio, const_folio, logops_folio, len_folio = extract_info(folio_ans, True)

    #llm
    dif_preds_llm, pred_count_llm = total_apps(pred_llm, True)
    dif_const_llm, const_count_llm = total_apps(const_llm, False)
    dif_logops_llm, logops_count_llm = total_apps(logops_llm, False)

    #folio
    dif_preds_folio, pred_count_folio = total_apps(pred_folio, True)
    dif_const_folio, const_count_folio = total_apps(const_folio, False)
    dif_logops_folio, logops_count_folio = total_apps(logops_folio, False)

    #Absolute Values
    dif_preds = abs(dif_preds_llm - dif_preds_folio)
    tot_aps_preds = abs(pred_count_llm - pred_count_folio)
    
    dif_const = abs(dif_const_llm - dif_const_folio)
    tot_aps_const = abs(const_count_llm - const_count_folio)

    dif_logops = abs(dif_logops_llm - dif_logops_folio)
    tot_aps_logops = abs(logops_count_llm - logops_count_folio)

    dif_premises = abs(len_llm - len_folio)

    logicsim_plus = dif_preds + tot_aps_preds + dif_const + tot_aps_const + dif_logops + tot_aps_logops + dif_premises
    return logicsim_plus

In [3]:
print("==================")
print("====始めましょう====")
print("==================")

gpt_vs_qwen = []
gpt_vs_folio = []
qwen_vs_folio = []

for i in range(len(tran_ds)):
    gpt = tran_ds["gpt Translations"][i]
    qwen = tran_ds["qwen Translations"][i]
    folio = tran_ds["premises-FOL"][i]

    gptqwen = logic_sim_plus_individual(gpt, qwen)
    gptfolio = logic_sim_plus_individual(gpt, folio)
    qwenfolio = logic_sim_plus_individual(qwen, folio)
    print("GPT vs Qwen: {}".format(gptqwen))
    print("GPT vs FOLIO: {}".format(gptfolio))
    print("FOLIO vs Qwen: {}".format(qwenfolio))

    if gptqwen < 6:
        print("GPT vs QWEN < 6")
        print('-------')
        print(gpt)
        print('-------')
        print(qwen)
        print('-------')
        print(folio)
        print("================")
    
    if gptfolio > 25 or qwenfolio > 25:
        print("LLM vs FOLIO > 25")
        print('-------')
        print(gpt)
        print('-------')
        print(qwen)
        print('-------')
        print(folio)
        print("================")

    gpt_vs_qwen.append(gptqwen)
    gpt_vs_folio.append(gptfolio)
    qwen_vs_folio.append(qwenfolio)
    print('-------------------')

====始めましょう====
GPT vs Qwen: 5
GPT vs FOLIO: 19
FOLIO vs Qwen: 18
GPT vs QWEN < 6
-------
∀x (ProSoccerPlayer(x) → ¬PlayBasketball(x))
∀x (ProSoccerDefender(x) → ProSoccerPlayer(x))
∀x (ProCenterBack(x) → ProSoccerDefender(x))
∃x (Athlete(x) ∧ ProCenterBack(x))

-------


∀x (ProfessionalSoccerPlayer(x) → ¬ProfessionalBasketballPlayer(x))  
∀x (ProfessionalSoccerDefender(x) → ProfessionalSoccerPlayer(x))  
∀x (ProfessionalCenterback(x) → ProfessionalSoccerDefender(x))  
∃x (Athlete(x) ∧ ProfessionalCenterback(x))  
ProfessionalBasketballPlayer(StephenCurry)
-------
∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Play(x, professionalBasketball))
∀x ((Professional(x) ∧ Defender(x)) → (Professional(x) ∧ SoccerPlayer(x)))
∀x ((Professional(x) ∧ CenterBack(x)) → (Professional(x) ∧ Defender(x)))
∃x (Athlete(x) ∧ Professional(x) ∧ CenterBack(x))
Play(stephenCurry, professionalBasketball)
-------------------
GPT vs Qwen: 5
GPT vs FOLIO: 19
FOLIO vs Qwen: 18
GPT vs QWEN < 6
-------
∀x (SoccerPlayer(x

In [4]:
import statistics

print("LogicSim promedio (GPT vs Qwen): {}".format(round(sum(gpt_vs_qwen)/len(gpt_vs_qwen), 3)))
print("Desviación estándar: {}".format(round(statistics.stdev(gpt_vs_qwen), 3)))
print('----')
print("LogicSim promedio (GPT vs FOLIO): {}".format(round(sum(gpt_vs_folio)/len(gpt_vs_folio), 3)))
print("Desviación estándar: {}".format(round(statistics.stdev(gpt_vs_folio), 3)))
print('----')
print("LogicSim promedio (FOLIO vs Qwen): {}".format(round(sum(qwen_vs_folio)/len(qwen_vs_folio), 3)))
print("Desviación estándar: {}".format(round(statistics.stdev(qwen_vs_folio), 3)))

LogicSim promedio (GPT vs Qwen): 12.973
Desviación estándar: 12.606
----
LogicSim promedio (GPT vs FOLIO): 23.721
Desviación estándar: 13.468
----
LogicSim promedio (FOLIO vs Qwen): 23.022
Desviación estándar: 15.027


In [ ]:
# FOL to Prover9 Syntax

def switch_quantifiers(text, cuantifier):
    """
        Elimina todos los cuantificadores y los traduce a sintaxis de Prover9. Sin importar la variable ni la cantidad de apariciones. Qué pedo soy una verga para esto.

        text = str ;  Texto a modificar.
        cuantifier = str ('forall', 'exists') ; Cuantificador a modificar.
    """
    if cuantifier == 'forall':
        regex = '∀[A-z]'
        cuant = 'all '
    else:
        regex = '∃[A-z]'
        cuant = 'exists '

    owo = re.finditer(regex, text)
    aux_list = list(owo)
    if len(aux_list) == 0:
        return text

    # Redefinimos el iterador porque hacer lista de un iterador lo consume. CHINGA TU MADRE PYTHON. VETE A LA BURGER.    
    owo = re.finditer(regex, text)
    temporal_str = ''

    for _ in owo:
        if temporal_str == '':
            temporal_str = text[0:_.start()] + cuant + _.group()[-1] + '.' + text[_.end():]
        else:
            value = re.search(regex, temporal_str)
            temporal_str = temporal_str[0:(value.start())] + cuant + value.group()[-1] + '.' + temporal_str[value.end():]
    
    return temporal_str



def fol_to_prover9(value):
    """
        Modifica los símbolos lógicos normales y los cambia por los valores adecuados para Prover9.*
    """
    temp = value.lower()
    temp = switch_quantifiers(temp, 'forall')
    temp = switch_quantifiers(temp, 'exists')
    temp = re.sub('-', '', temp)
    temp = re.sub('¬', '-', temp) 
    temp = re.sub('→|→', '-> ', temp)
    temp = re.sub('∧', '&', temp)
    temp = re.sub('∨', '|', temp)
    temp = re.sub('↔', '<->', temp)
    temp = re.sub('≠', '!=', temp)
    temp = re.sub(r'\'', '', temp)
    temp = re.sub(r'\[', '(', temp)
    temp = re.sub(r'\]', ')', temp)
    temp = re.sub('∴', '', temp)
    temp = re.sub('(\. \()', '.(', temp)
    temp = re.sub('\.{2}', '.', temp)
    temp = re.sub(r'\)\.', ')', temp)
    temp = re.sub(r'([^a-z]\.\()', '(', temp)
    return temp


def dash_predicates(text):
    """
        Cambia los predicados de la forma "texto-texto-texto(x)" -> "textotextotexto(x)"

        text = str ; el hilo a modificar.
    """
    all_values = len(re.findall(r'[a-z0-9]+(\-[a-z0-9]+\-{0,})+[a-z0-9]+', text))

    if all_values == 0:
        return text
    
    new_text = text
    for i in range(all_values):
        current_regex = re.search(r'[a-z0-9]+(\-[a-z0-9]+\-{0,})+[a-z0-9]+', new_text)
        split = current_regex.group().split()
        aux_text = ''
        for elem in split:
            aux_text = aux_text + elem
        new_text = new_text[:current_regex.start()] + aux_text + new_text[current_regex.end():]

    return new_text


def elim_spaces(text):
    """
        Elimina los espacios entre variables: lionel messi -> lionelmessi

        text = str; El texto a modificar.


        OBS: Este formato de funciones (Encontrar cantidades y luego iterar sobre las cantidades) me gusta bastante.
    """
    total_iters = len(list(re.finditer('([A-z]+ )+([A-z]{2,})', text)))
    if total_iters == 0:
        return text

    new_text = text
    for i in range(total_iters):
        current_regex = re.search('([A-z]+ )+([A-z]{2,})', new_text)
        split = current_regex.group().split()
        aux_text = ''
        for elem in split:
            aux_text = aux_text + elem
        new_text = new_text[:current_regex.start()] + aux_text + new_text[current_regex.end():]

    return new_text


# El XOR me tiene hasta los huevos cabrón te lo juro.
def xor_bonito(expression):
    """
        Elimina el símbolo de XOR, y lo reescribe en la fórmula (A OR B) AND NOT(A AND B)
    """
    individual_values = re.findall('([A-z|_|0-9]{2,}|¬)', expression)
    a = individual_values[0] + '(x)'
    b = individual_values[1] + '(x)'
    a_or_b = '(' + a + ' | ' + b + ')'
    not_a_and_b = '-(' + a + ' & ' + b +')'
    xor = a_or_b + ' & ' + not_a_and_b
    return xor

def xor_bonito_extreme(expression):
    """
        Elimina los XOR de fórmulas compuestas.
    """
    aux2 = re.search(r'([a-z]+\([a-z, ]+\)) ⊕ ([a-z]+\([a-z, ]+\))', expression)
    split = aux2.group().split('⊕')
    a_f = split[0]
    b_f = split[-1]
    a_or_b = '(' + a_f + ' | ' + b_f + ')'
    not_a_and_b = '-(' + a_f + ' & ' + b_f +')'
    xor = a_or_b + ' & ' + not_a_and_b
    return xor

def rewrite_xor(re_search, element, comp):
    """
        Genera una nueva expresión a partir del xor bonito. 

        re_search = re.search(regex, str)
        element = str ; same str as above
        comp = bool ; True iff predicates have multiple variables.
    """
    start = re_search.start()
    end = re_search.end()

    xor_substr = element[start:end]
    if comp:
        xor_chido = xor_bonito_extreme(xor_substr)
    else:
        xor_chido = xor_bonito(xor_substr)

    nuevo = element[:start] + xor_chido + element[end:]
    return nuevo


def clean(value):
    """
        Procesa una respuesta individual de FOLIO/GPT_TRANS/QWEN_TRANS para que se pase al formato de Prover9.
    """

    clean_premises_aux = value.split('\n')
    clean_premises = []
    for _ in clean_premises_aux:
        if _ != '':
            clean_premises.append(_)

    if "Premises" in clean_premises[0]:
        del clean_premises[0]

    for _ in clean_premises:
        clean_premises[clean_premises.index(_)] = re.sub('(:::)+([ A-z.]+)', '', _)

    for _ in clean_premises:
        clean_premises[clean_premises.index(_)] = re.sub('[0-9]\.', '', _)

    for instance in clean_premises:
        clean_premises[clean_premises.index(instance)] = fol_to_prover9(instance)

    for instance in clean_premises:
        clean_premises[clean_premises.index(instance)] = elim_spaces(instance)

    try:
        # Filtro XOR sencillo
        for element in clean_premises:
            xor_count = len(re.findall('⊕', element))
            if xor_count > 0:
                value = element
                while xor_count > 0:
                    aux = re.search('(-{0,1}[a-z]+\([a-z, ]+\)) ⊕ (-{0,1}[a-z]+\([a-z, ]+\))', element)
                    element = rewrite_xor(aux, element, False)
                    xor_count = len(re.findall('⊕', element))
                clean_premises[clean_premises.index(value)] = element

        # Filtro XOR multipremisa
        for element in clean_premises:
            xor_count = len(re.findall('⊕', element))
            if xor_count > 0:
                value = element
                while xor_count > 0:
                    aux = re.search('(-{0,1}[a-z]+\([a-z, ]+\)) ⊕ (-{0,1}[a-z]+\([a-z, ]+\))', element)
                    element = rewrite_xor(aux, element, True)
                    xor_count = len(re.findall('⊕', element))
                clean_premises[clean_premises.index(value)] = element

    except:
        print("Hubo un Xor malo")    

    return clean_premises

In [ ]:
for i in range(len(tran_ds)):
    print("=============================")
    print("============={}==============".format(i))
    print("=============================")
    print(tran_ds['premises-FOL'][i])
    print('-----------------')
    print(clean(tran_ds['premises-FOL'][i]))
    print(clean(tran_ds['qwen Translations'][i]))
    print(clean(tran_ds['gpt Translations'][i]))

=============0==============
∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Play(x, professionalBasketball))
∀x ((Professional(x) ∧ Defender(x)) → (Professional(x) ∧ SoccerPlayer(x)))
∀x ((Professional(x) ∧ CenterBack(x)) → (Professional(x) ∧ Defender(x)))
∃x (Athlete(x) ∧ Professional(x) ∧ CenterBack(x))
Play(stephenCurry, professionalBasketball)
-----------------
['all x.((professional(x) & soccerplayer(x)) ->  -play(x, professionalbasketball))', 'all x.((professional(x) & defender(x)) ->  (professional(x) & soccerplayer(x)))', 'all x.((professional(x) & centerback(x)) ->  (professional(x) & defender(x)))', 'exists x.(athlete(x) & professional(x) & centerback(x))', 'play(stephencurry, professionalbasketball)']
['all x.(professionalsoccerplayer(x) ->  -professionalbasketballplayer(x))  ', 'all x.(professionalsoccerdefender(x) ->  professionalsoccerplayer(x))  ', 'all x.(professionalcenterback(x) ->  professionalsoccerdefender(x))  ', 'exists x.(athlete(x) & professionalcenterback(x))  ', 

In [7]:
def check_arg_validity(index, llm):
    """
        Dadas premisas en lenguaje lógico, verifica que la conclusión correspondiente sea inferible.
    """
    llm_valid, llm_invalid, fol_valid, fol_invalid = 0, 0, 0, 0
    llm_value = tran_ds['{} Translations'.format(llm)][index]
    folio_value = tran_ds['premises-FOL'][index]

    true_validity = tran_ds['label'][index]
    print("Logical Validity: {}".format(true_validity))

    clean_llm = clean(llm_value)
    clean_folio = clean(folio_value)
    clean_conc = clean(tran_ds['conclusion-FOL'][index])
    
    args_llm = (
        clean_conc[0],
        clean_llm
    )

    args_folio = (
        clean_conc[0],
        clean_folio
    )
    
    # Lo que hacemos aquí es evaluar una conclusión contra las premisas de un LLM o del conjunto de datos.
    # OBS: A veces la premisa NO debe de ser deducible. 
    # ¿Cómo vamos a cuantificar este show?
    # Casos:
    # Premisas: TRUE, UNCERTAIN, FALSE.
    # Posibles resultados: TRUE, FALSE. ¿Cómo lidiamos con Uncertain? -> En estos casos Prover9 te regresa False. 
    
    print("LLM ({}):".format(llm))
    try:
        print("Ejecutable. Valor: {}".format(prove(args_llm)))
        llm_valid += 1
    except Exception as e:
        print(e)
        print(type(e))
        llm_invalid += 1
        
    print("FOLIO:")
    try:
        print("Ejecutable. Valor: {}".format(prove(args_folio)))
        fol_valid += 1
    except Exception as e:
        print(e)
        print(type(e))
        fol_invalid += 1
    print("======================")

    return llm_valid, llm_invalid, fol_valid, fol_invalid
    

In [8]:
_llm_valid, _llm_invalid, _fol_valid, _fol_invalid = 0,0,0,0
for i in range(len(tran_ds)):
    print(i)
    try:
        a, b, c, d = check_arg_validity(i, 'qwen')
        _llm_valid += a
        _llm_invalid += b
        _fol_valid += c
        _fol_invalid += d
    except:
        print("oops")

print("Premisas de LLM válidas: {}".format(_llm_valid))
print("Premisas de LLM inválidas: {}".format(_llm_invalid))
print("Premisas de FOLIO válidas: {}".format(_fol_valid))
print("Premisas de FOLIO inválidas: {}".format(_fol_invalid))


0
Logical Validity: Uncertain
LLM (qwen):
Ejecutable. Valor: False
FOLIO:
Ejecutable. Valor: False
1
Logical Validity: False
LLM (qwen):
Ejecutable. Valor: False
FOLIO:
Ejecutable. Valor: False
2
Logical Validity: True
LLM (qwen):
Ejecutable. Valor: False
FOLIO:
Ejecutable. Valor: True
3
Logical Validity: Uncertain
LLM (qwen):
Ejecutable. Valor: False
FOLIO:
Ejecutable. Valor: False
4
Logical Validity: True
Hubo un Xor malo
LLM (qwen):
Unexpected token: '⊕'.  Expected token ')'.
-(stayalonein(tyler, highschool) ⊕ (welladjustedwithin(x, community) & engagedwithin(x, community))) ->  -(manyopportunitytoimprove(tyler, communicationskill) ⊕ stayalonein(tyler, highschool))
                                 ^
<class 'nltk.sem.logic.LogicalExpressionException'>
FOLIO:
Unexpected token: '⊕'.  Expected token ')'.
-(stayalonein(tyler, highschool) ⊕ (welladjustedwithin(x, community) & engagedwithin(x, community))) ->  -(manyopportunitytoimprove(tyler, communicationskill) ⊕ stayalonein(tyler, highs

In [ ]:
# Vamos a tomarnos una pausa de la traducción a la sintaxis de Prover9.
# Todavía falta por ver unos casos tipo: e(x) y mejorar el hdspm del XOR.
# Ahora vamos a generar las respuestas del modelo para el proceso de inferencia.